In [2]:
from datasets import load_dataset, Dataset
from bs4 import BeautifulSoup
import re

## Uploading Noisy Dataset

In [6]:
dataset = load_dataset('DATA')

text=[]

for i in dataset['train']:
    xml = i['audio']['path'].replace('.wav', '.xml')
    transcript = []
    with open(xml, 'r') as f:
        content = f.read()
        Bs_data = BeautifulSoup(content, "xml")
        b_unique = Bs_data.find_all('text')
        for j in b_unique:
            transcript.append(j.text)

    text.append(transcript)

for j in range(len(text)):
    for i in range(len(text[j])):
        transcription = text[j][i]
        transcription = re.sub(r'\[.*?\]', '', transcription)
        transcription = transcription.replace('[', '').replace(']', '')

        transcription = re.sub(r'\s+', ' ', transcription)

        text[j][i] = transcription
    

dataset['train'] = dataset['train'].add_column(name='text', column=text)

for i in range(len(text)):
    transcription = ''

    for j in text[i]:
        transcription += j + ' '

    if transcription.endswith(' '):
        transcription = transcription[:-1]

    text[i] = transcription

dataset['train'] = dataset['train'].add_column(name='text Str', column=text)

dataset = dataset['train'].train_test_split(test_size=0.2)

dataset.push_to_hub('rodoggx/ATCO2-ASR-1h')

Resolving data files:   0%|          | 0/560 [00:00<?, ?it/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/448 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/112 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/533 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/rodoggx/ATCO2-ASR-1h/commit/a5e8b5a2b16688023ac2e51c6e47db2377ce358d', commit_message='Upload dataset', commit_description='', oid='a5e8b5a2b16688023ac2e51c6e47db2377ce358d', pr_url=None, pr_revision=None, pr_num=None)